## Lora Model Training by PEFT

PEFT: Parameters-Efficient Fine-Tuning 

- Supported Models:
    - CasualLM
    - Seq2SeqLM
    - SequenceClassification
    - TokenClassification
    
- Supported PEFT Methods:
    - PromptEmbedding
    - PromptEncoder
    - PrefixTuning
    - Lora

In [1]:
from transformers import AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

/home/duhu/anaconda3/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lora

- lora仅支持`nn.Linear`，所以需要确定要使用lora的层的属性；

### LoraConfig

输入的参数：
- r: lora attention dimesions, default: 8
- taget_modules: str or list of string, the name of the modules to apply lora to. 默认不指定module，如果要指定的话，可以输入base model的module name，这个需要自己打印出来base model的结构才能确定是啥；不指定的时候会在内置的一个字典里面去找，这里包含了市面上的大部分常用的预训练模型，以及其对应的可以低秩分解：
```json
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING = {
    "t5": ["q", "v"],
    "mt5": ["q", "v"],
    "bart": ["q_proj", "v_proj"],
    "gpt2": ["c_attn"],
    "bloom": ["query_key_value"],
    "blip-2": ["q", "v", "q_proj", "v_proj"],
    "opt": ["q_proj", "v_proj"],
    "gptj": ["q_proj", "v_proj"],
    "gpt_neox": ["query_key_value"],
    "gpt_neo": ["q_proj", "v_proj"],
    "bert": ["query", "value"],
    "roberta": ["query", "value"],
    "xlm-roberta": ["query", "value"],
    "electra": ["query", "value"],
    "deberta-v2": ["query_proj", "value_proj"],
    "deberta": ["in_proj"],
    "layoutlm": ["query", "value"],
    "llama": ["q_proj", "v_proj"],
    "chatglm": ["query_key_value"],
}
```
- lora_alpha: the alpha paramter for Lora scaling **Explanation Later**
- lora_dropout: the dropout of Lora
- fan_in_fan_out: Set this to True if the layer to replace stores weight like (fan_in, fan_out) ***EL**
- bias: Bias type for lora, 'none', 'all' or 'lora_only'
- module_to_save: List of modules apart from LoRA layers to be set as trainable and saved in the final checkpoint.除了涉及到lora的部分，剩下的还有哪些模块需要参与训练，最后也会保存到最终的checkpoint里，比如说用预训练模型去做一个分类任务，最后一层（分类层）的参数也是需要训练和保存的。

内嵌的属性：
- peft_type: PeftType.Lora 

### ~~GPT-2 as an example~~ 

GPT-2的attention中的q/k/v的转换用的不是线性层，而是conv1d……

### ~~GPT as an example~~

### EleutherAI/gpt-neo-1.3B

In [2]:
pretrained_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [3]:
pretrained_model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj):

In [4]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, # 不指定任务模型的类别的时候，返回的是PeftModel
    r=8, 
    lora_alpha=16, 
    lora_dropout=.1, 
    target_modules=["q_proj", "k_proj", "v_proj"])

- 此处我们指定的任务类别是`CAUSAL_LM`，所以返回的model是`PeftModelForCausalLM`
- `PeftModelForCausalLM`是继承自`PeftModel`的
- `PeftModel`封装了通用的一些方法，`PeftModelForCausalLM`定制化一些专对CausalLM的一些方法；
- `PeftModel`有个初始化参数，`adapter_name`，默认值是`default`，这个参数是为了基于同一个base model搞出来多个adapter，想要切换adapter，可以直接根据这个参数来切换，具体的逻辑和作用可以参考[Multi Adapter support](https://github.com/huggingface/peft/pull/263#issue-1654639358)。主要目的就是：use multi-adapter at same time.

`Peft`的一些方法：
- 如果Peft模型的类别是Lora的话，PeftModel类的初始化方法里面的base_model是LoraModel，PROMPT_TUNING/P_TUNING/PREFIX_TUNING的base model还是预训练的模型；
- set_additional_trainable_modules(self, peft_config, adapter_name)
    - `PeftConfig`中的`modules_to_save`不为None的时候(比如基于base model训练一个分类器，最后一层的module应该是要保存的)，就把这些modules添加到self.modules_to_save，同时设定这些modules可训练； 比如分类器中，modules_to_save的设定为
    ```python
    self.modules_to_save = {"classifier", "score"}
    ```
- add_adapter(self, adapter_name, peft_config)
    - 如果`PeftConfig`的基类是`PromptLearningConfig`(也就是PROMPT_TUNING/P_TUNING/PREFIX_TUNING这些)，就需要设定prompt encoder，同时调用`set_additional_trainable_modules`
    - 只有PROMPT_TUNING/P_TUNING/PREFIX_TUNING才有prompt encoder

In [5]:
model = get_peft_model(pretrained_model, lora_config)

In [6]:
model.print_trainable_parameters()  # model == pretrained_model

trainable params: 2359296 || all params: 1317935104 || trainable%: 0.17901458067543818


PeftModelForCausalLM
- 继承自`PeftModel`
- 初始化：
    ```python
    self.base_model_prepare_inputs_for_generation = self.base_model.prepare_inputs_for_generation
    ```
    从base model中可以看出来，这个函数返回的是
    ```json
    {
            "input_ids": input_ids,
            "past_key_values": past_key_values,
            "use_cache": kwargs.get("use_cache"),
            "position_ids": position_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
     }

    ```
    这个函数好像只有在生成数据的时候使用，**EL**
- 如果PeftConfig是Lora及其变体的话，base model是已经加入了初始化的low-rank decomposition weights（A/B矩阵）的LoraModel了，此时，预训练模型的部分参数已经被冻结；

LoraModel
- lora模型的forward和base model的forward是一样的，因为最终参与计算的这些权重是$$W = W_{0} + AB$$
- Lora支持nn.Linear/nn.Embedding/nn.Conv1D
- Lora的核心模块是`add_adapter`：
    - self._prepare_lora_config(config, model_config)：
        这个函数做了两个事情：
        - 确定要做低秩分解的target_modules：配置参数指定，则根据指定来，如果没有指定，那么就按照内置的常用的模型来；
        - 确定是否inference_mode，如果是，就合并权重；
    - self._find_and_replace(adapter_name):
        这个函数核心也就做了两个事情：
        - find：找到需要加入lora的module，假设module为nn.Linear的话，生成与其对应的模块Linear，这个模块继承自nn.Linear和LoraLayer，nn.Linear与预训练中要被替换的线性层模块shape完全一致；LoraLayer生成对应配置的lora模块
        - replace：将生成的模块Linear替换掉原来预训练模型中的nn.Linear，同时将原来预训练模型的nn.Linear的权重赋值给Linear的nn.Linear的模块；利用`setattr`将原来的模块替换成新模块。
        

In [7]:
print(str(model.base_model)[:20])  # AutoModelForCausalLM + Lora => LoraModel

LoraModel(
  (model)


In [8]:
print(model.peft_config)

{'default': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, base_model_name_or_path='EleutherAI/gpt-neo-1.3B', task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['q_proj', 'k_proj', 'v_proj'], lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True)}


In [9]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoForCausalLM(
      (transformer): GPTNeoModel(
        (wte): Embedding(50257, 2048)
        (wpe): Embedding(2048, 2048)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPTNeoBlock(
            (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (attn): GPTNeoAttention(
              (attention): GPTNeoSelfAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
                (resid_dropout): Dropout(p=0.0, inplace=False)
                (k_proj): Linear(
                  in_features=2048, out_features=2048, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
       

In [10]:
from transformers import GPT2Tokenizer

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [4]:
tokenizer.bos_token

'<|endoftext|>'

In [5]:
tokenizer.eos_token

'<|endoftext|>'

In [6]:
print(tokenizer.bos_token_id)

50256


In [7]:
tokenizer.convert_ids_to_tokens(21106)

'Below'

In [11]:
tokenizer.pad_token

Using pad_token, but it is not set yet.
